# WeatherPy

---

## Starter Code to Generate Random Geographic Coordinates and a List of Cities

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
from scipy.stats import linregress

# Import the OpenWeatherMap API key
from api_keys import weather_api_key

# Import citipy to determine the cities based on latitude and longitude
from citipy import citipy

### Generate the Cities List by Using the `citipy` Library

In [ ]:
# Empty list for holding the latitude and longitude combinations
lat_lngs = []

# Empty list for holding the cities names
cities = []

# Range of latitudes and longitudes
lat_range = (-90, 90)
lng_range = (-180, 180)

# Create a set of random lat and lng combinations
lats = np.random.uniform(lat_range[0], lat_range[1], size=1500)
lngs = np.random.uniform(lng_range[0], lng_range[1], size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name

    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
print(f"Number of cities in the list: {len(cities)}")

---

## Requirement 1: Create Plots to Showcase the Relationship Between Weather Variables and Latitude

### Use the OpenWeatherMap API to retrieve weather data from the cities list generated in the started code

In [ ]:
# Set the API base URL
url = "http://api.openweathermap.org/data/2.5/weather?"

# Define an empty list to fetch the weather data for each city
city_data = []

# Print to logger
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters
record_count = 1
set_count = 1

# Loop through all the cities in our list to fetch weather data
for i, city in enumerate(cities):

    # Group cities in sets of 50 for logging purposes
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 0

    # Create endpoint URL with each city
    city_url = f"{url}?q={city}&appid={weather_api_key}&units=metric"

    # Log the url, record, and set numbers
    print("Processing Record %s of Set %s | %s" % (record_count, set_count, city))

    # Add 1 to the record count
    record_count += 1

    # Run an API request for each of the cities
    try:
        # Parse the JSON and retrieve data
        city_weather = requests.get(city_url).json()

        # Parse out latitude, longitude, max temp, humidity, cloudiness, wind speed, country, and date
        city_lat = city_weather['coord']['lat']
        city_lng = city_weather['coord']['lon']
        city_max_temp = city_weather['main']['temp_max']
        city_humidity =  city_weather['main']['humidity']
        city_clouds = city_weather['clouds']['all']
        city_wind = city_weather['wind']['speed']
        city_country = city_weather['sys']['country']
        city_date = city_weather['dt']

        # Append the City information into city_data list
        city_data.append({"City": city,
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

    # If an error is experienced, skip the city
    except:
        print("City not found. Skipping...")
        pass

    # pause to avoid rate limiting
    time.sleep(1)

# Indicate that Data Loading is complete
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

In [ ]:
# Convert the cities weather data into a Pandas DataFrame
city_data_df =  pd.DataFrame(city_data)

# Show Record Count
city_data_df.count()

In [ ]:
# Display sample data
city_data_df.head()

In [ ]:
# Export the City_Data into a csv
city_data_df.to_csv("output_data/cities.csv", index_label="City_ID")

In [ ]:
# Read saved data
city_data_df = pd.read_csv("output_data/cities.csv", index_col="City_ID")

# Display sample data
city_data_df.head()

### Create the Scatter Plots Requested

#### Latitude Vs. Temperature

In [ ]:
# Build scatter plot for latitude vs. temperature
plt.figure(figsize=(10,6))
plt.scatter(city_data_df['Lat'], city_data_df['Max Temp'], color='blue', edgecolors='black')

# Incorporate the other graph properties
plt.title('City Max Latitude vs. Temperature (2024-06-17)', fontsize=14)
plt.xlabel('Latitude', fontsize=12)
plt.ylabel('Temperature (°C)', fontsize=12)
plt.grid(True)

# Save the figure
plt.savefig("output_data/Fig1.png")

# Show plot
plt.show()

#### Latitude Vs. Humidity

In [ ]:
# Build the scatter plots for latitude vs. humidity
plt.figure(figsize=(10,6))
plt.scatter(city_data_df['Lat'], city_data_df['Humidity'], color='blue', edgecolors='black')

# Incorporate the other graph properties
plt.title('City Latitude vs. Humidity (2024-06-17)', fontsize=14)
plt.xlabel('Latitude', fontsize=12)
plt.ylabel('Humidity (%)', fontsize=12)
plt.grid(True)

# Save the figure
plt.savefig("output_data/Fig2.png")

# Show plot
plt.show()

#### Latitude Vs. Cloudiness

In [ ]:
# Build the scatter plots for latitude vs. cloudiness
plt.figure(figsize=(10,6))
plt.scatter(city_data_df['Lat'], city_data_df['Cloudiness'], color='blue', edgecolors='black')

# Incorporate the other graph properties
plt.title('City Latitude vs. Cloudiness (2024-06-17)', fontsize=14)
plt.xlabel('Latitude', fontsize=12)
plt.ylabel('Cloudiness (%)', fontsize=12)
plt.grid(True)

# Save the figure
plt.savefig("output_data/Fig3.png")

# Show plot
plt.show()

#### Latitude vs. Wind Speed Plot

In [ ]:
# Build the scatter plots for latitude vs. wind speed
plt.figure(figsize=(10,6))
plt.scatter(city_data_df['Lat'], city_data_df['Wind Speed'], color='blue', edgecolors='black')


# Incorporate the other graph properties
plt.title('City Latitude vs. Wind Speed (2024-06-17)', fontsize=14)
plt.xlabel('Latitude', fontsize=12)
plt.ylabel('Wind Speed (m/s)', fontsize=12)
plt.grid(True)

# Save the figure
plt.savefig("output_data/Fig4.png")

# Show plot
plt.show()

---

## Requirement 2: Compute Linear Regression for Each Relationship


In [ ]:
# Define a function to create Linear Regression plots

def compute_and_print_regression(x, y, label):
    slope, intercept, r_value, _, _ = linregress(x, y)
    regression_values = slope * x + intercept
    plt.plot(x, regression_values, color='red')
    plt.scatter(x, y)
    plt.title(f'{label}: r={r_value:.2f}')
    plt.xlabel('Latitude')
    plt.ylabel(label.split(' vs. ')[-1])
    plt.show()

In [ ]:
# Create a DataFrame with the Northern Hemisphere data (Latitude >= 0)
northern_hemi_df = city_data_df[city_data_df['Lat'] >= 0]]

# Display sample data
northern_hemi_df.head()

In [ ]:
# Create a DataFrame with the Southern Hemisphere data (Latitude < 0)
southern_hemi_df = city_data_df[city_data_df['Lat'] < 0]]

# Display sample data
southern_hemi_df.head()

###  Temperature vs. Latitude Linear Regression Plot

In [ ]:
# Linear regression on Northern Hemisphere
northern_latitudes = northern_hemi_df['Lat']
northern_temperatures = northern_hemi_df['Max Temp']

# Perform linear regression on Northern Hemisphere data
slope, intercept, r_value, _, _ = linregress(northern_latitudes, northern_temperatures)

# Create regression values (line)
regress_values_north = northern_latitudes * slope + intercept

# Plot scatter plot with regression line
plt.figure(figsize=(10,6))
plt.scatter(northern_latitudes, northern_temperatures, color='blue', edgecolors='black', label="City Data (Northern Hemisphere)")
plt.plot(northern_latitudes, regress_values_north, color='red', label=f"Regression Line (r² = {r_value**2:.10f})")

# Add titles and labels
plt.title('Northern Hemisphere: Latitude vs. Max Temperature', fontsize=14)
plt.xlabel('Latitude', fontsize=12)
plt.ylabel('Max Temperature (°C)', fontsize=12)
plt.legend()
plt.grid(True)

# Show the r-squared value in the console
print(f"Northern Hemisphere r^2-value: {r_value**2}")

plt.show()

In [ ]:
# Linear regression on Southern Hemisphere
southern_latitudes = southern_hemi_df['Lat']
southern_temperatures = southern_hemi_df['Max Temp']

# Perform linear regression on Southern Hemisphere data
slope, intercept, r_value, _, _ = linregress(southern_latitudes, southern_temperatures)

# Create regression values line
regress_values_south = southern_latitudes * slope + intercept

# Plot scatter plot with regression line
plt.figure(figsize=(10,6))
plt.scatter(southern_latitudes, southern_temperatures, color='blue', edgecolors='black', label="City Data (Southern Hemisphere)")
plt.plot(southern_latitudes, regress_values_south, color='red', label=f"Regression Line (r² = {r_value**2:.10f})")

# Add titles and labels
plt.title('Southern Hemisphere: Latitude vs. Max Temperature', fontsize=14)
plt.xlabel('Latitude', fontsize=12)
plt.ylabel('Max Temperature (°C)', fontsize=12)
plt.legend()
plt.grid(True)

# Show the r-squared value in the console
print(f"Southern Hemisphere r^2-value: {r_value**2}")

plt.show()

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Humidity vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
northern_humidity = northern_hemi_df['Humidity']

# Perform linear regression on Northern Hemisphere data (Humidity vs. Latitude)
slope, intercept, r_value, _, _ = linregress(northern_latitudes, northern_humidity)

# Create regression line
regress_values_north = northern_latitudes * slope + intercept

# Plot scatter plot with regression line
plt.figure(figsize=(10,6))
plt.scatter(northern_latitudes, northern_humidity, color='blue', edgecolors='black', label="City Data (Northern Hemisphere)")
plt.plot(northern_latitudes, regress_values_north, color='red', label=f"Regression Line (r² = {r_value**2:.10f})")

# Add titles and labels
plt.title('Northern Hemisphere: Latitude vs. Humidity', fontsize=14)
plt.xlabel('Latitude', fontsize=12)
plt.ylabel('Humidity (%)', fontsize=12)
plt.legend()
plt.grid(True)

# Show the r-squared value in the console
print(f"Northern Hemisphere Humidity r^2-value: {r_value**2}")

plt.show()

In [ ]:
# Southern Hemisphere
southern_humidity = southern_hemi_df['Humidity']

# Perform linear regression on Northern Hemisphere data (Humidity vs. Latitude)
slope, intercept, r_value, _, _ = linregress(southern_latitudes, southern_humidity)

# Create regression line
regress_values_north = southern_latitudes * slope + intercept

# Plot scatter plot with regression line
plt.figure(figsize=(10,6))
plt.scatter(southern_latitudes, southern_humidity, color='blue', edgecolors='black', label="City Data (Southern Hemisphere)")
plt.plot(southern_latitudes, regress_values_south, color='red', label=f"Regression Line (r² = {r_value**2:.10f})")

# Add titles and labels
plt.title('Southern Hemisphere: Latitude vs. Humidity', fontsize=14)
plt.xlabel('Latitude', fontsize=12)
plt.ylabel('Humidity (%)', fontsize=12)
plt.legend()
plt.grid(True)

# Show the r-squared value in the console
print(f"Southern Hemisphere Humidity r^2-value: {r_value**2}")

plt.show()

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Cloudiness vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
northern_cloudiness = northern_hemi_df['Cloudiness']

# Perform linear regression on Northern Hemisphere data (CLoudiness vs. Latitude)
slope, intercept, r_value, _, _ = linregress(northern_latitudes, northern_cloudiness)

# Create regression line
regress_values_north = northern_latitudes * slope + intercept

# Plot scatter plot with regression line
plt.figure(figsize=(10,6))
plt.scatter(northern_latitudes, northern_cloudiness, color='blue', edgecolors='black', label="City Data (Northern Hemisphere)")
plt.plot(northern_latitudes, regress_values_north, color='red', label=f"Regression Line (r² = {r_value**2:.10f})")

# Add titles and labels
plt.title('Northern Hemisphere: Latitude vs. Cloudiness', fontsize=14)
plt.xlabel('Latitude', fontsize=12)
plt.ylabel('Clouiness (%)', fontsize=12)
plt.legend()
plt.grid(True)

# Show the r-squared value in the console
print(f"Northern Hemisphere CLoudiness r^2-value: {r_value**2}")

plt.show()

In [ ]:
# Southern Hemisphere
southern_cloudiness = southern_hemi_df['Cloudiness']

# Perform linear regression on Northern Hemisphere data (CLoudiness vs. Latitude)
slope, intercept, r_value, _, _ = linregress(southern_latitudes, southern_cloudiness)

# Create regression line
regress_values_south = southern_latitudes * slope + intercept

# Plot scatter plot with regression line
plt.figure(figsize=(10,6))
plt.scatter(southern_latitudes, southern_cloudiness, color='blue', edgecolors='black', label="City Data (Southern Hemisphere)")
plt.plot(Southern_latitudes, regress_values_south, color='red', label=f"Regression Line (r² = {r_value**2:.10f})")

# Add titles and labels
plt.title('Southern Hemisphere: Latitude vs. Cloudiness', fontsize=14)
plt.xlabel('Latitude', fontsize=12)
plt.ylabel('Clouiness (%)', fontsize=12)
plt.legend()
plt.grid(True)

# Show the r-squared value in the console
print(f"Southern Hemisphere CLoudiness r^2-value: {r_value**2}")

plt.show()

**Discussion about the linear relationship:** YOUR RESPONSE HERE

### Wind Speed vs. Latitude Linear Regression Plot

In [ ]:
# Northern Hemisphere
northern_windspeed = northern_hemi_df['Wind Speed']

# Perform linear regression on Northern Hemisphere data (CLoudiness vs. Latitude)
slope, intercept, r_value, _, _ = linregress(northern_latitudes, northern_windspeed)

# Create regression line
regress_values_north = northern_latitudes * slope + intercept

# Plot scatter plot with regression line
plt.figure(figsize=(10,6))
plt.scatter(northern_latitudes, northern_windspeed, color='blue', edgecolors='black', label="City Data (Northern Hemisphere)")
plt.plot(northern_latitudes, regress_values_north, color='red', label=f"Regression Line (r² = {r_value**2:.10f})")

# Add titles and labels
plt.title('Northern Hemisphere: Latitude vs. Wind Speed', fontsize=14)
plt.xlabel('Latitude', fontsize=12)
plt.ylabel('Wind Speed (%)', fontsize=12)
plt.legend()
plt.grid(True)

# Show the r-squared value in the console
print(f"Northern Hemisphere Wind Speed r^2-value: {r_value**2}")

plt.show()

In [ ]:
# Southern Hemisphere
southern_windspeed = southern_hemi_df['Wind Speed']

# Perform linear regression on Northern Hemisphere data (CLoudiness vs. Latitude)
slope, intercept, r_value, _, _ = linregress(southern_latitudes, southern_windspeed)

# Create regression line
regress_values_south = southern_latitudes * slope + intercept

# Plot scatter plot with regression line
plt.figure(figsize=(10,6))
plt.scatter(southern_latitudes, southern_windspeed, color='blue', edgecolors='black', label="City Data (Southern Hemisphere)")
plt.plot(southern_latitudes, regress_values_south, color='red', label=f"Regression Line (r² = {r_value**2:.10f})")

# Add titles and labels
plt.title('Southern Hemisphere: Latitude vs. Wind Speed', fontsize=14)
plt.xlabel('Latitude', fontsize=12)
plt.ylabel('Wind Speed (%)', fontsize=12)
plt.legend()
plt.grid(True)

# Show the r-squared value in the console
print(f"Southern Hemisphere Wind Speed r^2-value: {r_value**2}")

plt.show()

**Discussion about the linear relationship:** YOUR RESPONSE HERE